### 1. Установить Mrec (или альтернативе, по договорённости с преподавателем)

In [73]:
cd data/rs

/home/mitya/projects/homework/data/rs


In [74]:
! ipcluster stop &2>/dev/null
! ipcluster start -n4 --daemonize
! rm -rf tmp && mkdir tmp

In [75]:
! rm -rf tmp/prepared && mkdir tmp/prepared
! mrec_prepare --dataset ml-100k/u.data --outdir tmp/prepared --rating_thresh 4 --test_size 0.3 --binarize --min_items_per_user=7
! ls -lah tmp/prepared

[2017-12-10 16:03:16,840] INFO: sorting input data...
[2017-12-10 16:03:16,938] INFO: creating split 0: /home/mitya/projects/homework/data/rs/tmp/prepared/u.data.train.0 /home/mitya/projects/homework/data/rs/tmp/prepared/u.data.test.0
[2017-12-10 16:03:17,254] INFO: creating split 1: /home/mitya/projects/homework/data/rs/tmp/prepared/u.data.train.1 /home/mitya/projects/homework/data/rs/tmp/prepared/u.data.test.1
[2017-12-10 16:03:17,568] INFO: creating split 2: /home/mitya/projects/homework/data/rs/tmp/prepared/u.data.train.2 /home/mitya/projects/homework/data/rs/tmp/prepared/u.data.test.2
[2017-12-10 16:03:17,881] INFO: creating split 3: /home/mitya/projects/homework/data/rs/tmp/prepared/u.data.train.3 /home/mitya/projects/homework/data/rs/tmp/prepared/u.data.test.3
[2017-12-10 16:03:18,194] INFO: creating split 4: /home/mitya/projects/homework/data/rs/tmp/prepared/u.data.train.4 /home/mitya/projects/homework/data/rs/tmp/prepared/u.data.test.4
[2017-12-10 16:03:18,511] INFO: cleaning 

In [76]:
! rm -rf tmp/models && mkdir tmp/models
! mrec_train -n4 --input_format tsv --train "tmp/prepared/u.data.train.*" --outdir tmp/models --model=slim --l1_reg=0.01

[2017-12-10 16:03:19,651] INFO: processing /home/mitya/projects/homework/data/rs/tmp/prepared/u.data.train.1...
[2017-12-10 16:03:19,652] INFO: finding number of items...
[2017-12-10 16:03:20,029] INFO: 943 users and 1682 items
[2017-12-10 16:03:20,029] INFO: creating sims directory /home/mitya/projects/homework/data/rs/tmp/models/u.data.train.1-sims...
[2017-12-10 16:03:20,032] INFO: checking for existing output sims...
[2017-12-10 16:03:20,033] INFO: creating tasks...
[2017-12-10 16:03:20,033] INFO: running 4 tasks in parallel across ipython engines...
[2017-12-10 16:03:22,077] INFO: checking output files...
[2017-12-10 16:03:22,078] INFO: SUCCESS: all tasks completed
[2017-12-10 16:03:22,078] INFO: concatenating 4 partial output files...
[2017-12-10 16:03:22,081] INFO: removing partial output files...
[2017-12-10 16:03:22,081] INFO: loading 1682 items in SLIM model from /home/mitya/projects/homework/data/rs/tmp/models/u.data.train.1.sims.tsv
[2017-12-10 16:03:22,195] INFO: done
[201

In [77]:
! rm -rf tmp/recs && mkdir tmp/recs
! mrec_predict --input_format tsv --test_input_format tsv --train "tmp/prepared/u.data.train.*" --modeldir tmp/models/ --outdir tmp/recs

[2017-12-10 16:03:33,678] INFO: processing /home/mitya/projects/homework/data/rs/tmp/prepared/u.data.train.1...
[2017-12-10 16:03:33,678] INFO: creating recs directory /home/mitya/projects/homework/data/rs/tmp/recs/u.data.train.1-recs...
[2017-12-10 16:03:33,681] INFO: checking for existing output recs...
[2017-12-10 16:03:33,681] INFO: creating tasks...
[2017-12-10 16:03:33,682] INFO: loading dataset to get size...
[2017-12-10 16:03:34,040] INFO: loading model to get size...
[2017-12-10 16:03:34,079] INFO: created 1 tasks, 287016 users per task
[2017-12-10 16:03:34,079] INFO: running in parallel across ipython engines...
[2017-12-10 16:03:35,046] INFO: checking output files...
[2017-12-10 16:03:35,046] INFO: SUCCESS: all tasks completed
[2017-12-10 16:03:35,046] INFO: concatenating 1 partial output files...
[2017-12-10 16:03:35,051] INFO: removing partial output files...
[2017-12-10 16:03:35,051] INFO: done
[2017-12-10 16:03:35,052] INFO: processing /home/mitya/projects/homework/data/

In [78]:
! head tmp/recs/u.data.train.0.recs.tsv

1	50	1.0100884654177793
1	423	0.46517269995272925
1	96	0.45665582031827967
1	168	0.4552554712989447
1	173	0.43120327922252144
1	475	0.42956109322882313
1	79	0.37197817889206664
1	22	0.3552901814508858
1	23	0.34935208851547844
1	655	0.34823913674277274


In [79]:
# save fixed result to repo
# prepared, models, recs
! rm -rf ../../results/mrec-4-2 && mkdir -p ../../results/mrec-4-2
! cp -R tmp/* ../../results/mrec-4-2

In [80]:
cd -

/home/mitya/projects/homework


### 2. Получить для MovieLens 100K рекомендации:

In [81]:
! ipcluster stop &2>/dev/null

In [82]:
import sys
import pandas as pd
import scipy
import numpy as np
from os import path
from scipy.sparse import csr_matrix

In [83]:
def read_csv(filename: str):
    data_dir = 'data/rs/ml-latest-small'
    data = pd.read_csv(path.join(data_dir, filename + ".csv"))
    return data

def sparse_info(sparse_matrix: csr_matrix):
    print("Размерности матрицы: {}".format(sparse_matrix.shape))
    print("Ненулевых элементов в матрице: {}".format(sparse_matrix.nnz))
    print("Доля ненулевых элементов: {}"
          .format(sparse_matrix.nnz / sparse_matrix.shape[0] / sparse_matrix.shape[1])
    )
    print("Среднее значение ненулевых элементов: {}".format(sparse_matrix.data.mean()))
    print("Максимальное значение ненулевых элементов: {}".format(sparse_matrix.data.max()))
    print("Минимальное значение ненулевых элементов: {}".format(sparse_matrix.data.min()))

In [84]:
ratings = read_csv("ratings")
movies = read_csv("movies")
ratings["movie_id"] = ratings["movieId"].astype("category").cat.codes.copy() + 1

last_movie_id = ratings["movie_id"].max()
last_user_id = ratings["userId"].max()
mean_rating = ratings["rating"].mean()

In [85]:
user_x_item = ratings[["userId", "movie_id"]].as_matrix()
user_item_matrix = csr_matrix(
    (
        (ratings["rating"] - mean_rating).tolist(),
        (
            [pair[0] for pair in user_x_item],
            [pair[1] for pair in user_x_item],
        )
    ),
    shape=(last_user_id + 1, last_movie_id + 1),
    dtype=np.float32
)

sparse_info(user_item_matrix)

Размерности матрицы: (672, 9067)
Ненулевых элементов в матрице: 100004
Доля ненулевых элементов: 0.01641286822438251
Среднее значение ненулевых элементов: -8.544580154534742e-09
Максимальное значение ненулевых элементов: 1.45639169216156
Минимальное значение ненулевых элементов: -3.0436081886291504


#### по популярности

In [86]:
from scipy.spatial.distance import pdist, squareform

similarity_matrix = squareform(
    1 - pdist(
        user_item_matrix.todense().T,
        metric="cosine"
    )
)

print(similarity_matrix.shape)

(9067, 9067)


In [87]:
movies.columns, ratings.columns

(Index(['movieId', 'title', 'genres'], dtype='object'),
 Index(['userId', 'movieId', 'rating', 'timestamp', 'movie_id'], dtype='object'))

In [88]:
for ind, row in movies.iterrows():
    jnd = row.title.find('Pulp Fiction')
    if jnd >= 0:
        r = ratings[ratings.movieId==row.movieId].iloc[0]
        print(int(r.movie_id))

267


In [89]:
from scipy.spatial import distance

def get_similar_movies(movie_id: int, n: int) -> list:
    return (-similarity_matrix[movie_id, :]).argsort()[:n]

similar_movies = get_similar_movies(267, 10)
movies[movies["movieId"].isin(
    ratings[ratings["movie_id"].isin(similar_movies)]["movieId"].tolist()
)]

,movieId,title,genres
45,47,Seven (a.k.a. Se7en) (1995),Mystery|Thriller
48,50,"Usual Suspects, The (1995)",Crime|Mystery|Thriller
284,318,"Shawshank Redemption, The (1994)",Crime|Drama
525,593,"Silence of the Lambs, The (1991)",Crime|Horror|Thriller
535,608,Fargo (1996),Comedy|Crime|Drama|Thriller
695,858,"Godfather, The (1972)",Crime|Drama
880,1089,Reservoir Dogs (1992),Crime|Mystery|Thriller
969,1213,Goodfellas (1990),Crime|Drama
2288,2858,American Beauty (1999),Drama|Romance
2374,2959,Fight Club (1999),Action|Crime|Drama|Thriller


#### с помощью kNN

In [90]:
from sklearn.preprocessing import normalize, binarize

def get_cosine_similarity_matrix(user_item_matrix):
    user_item_csr = user_item_matrix.tocsr()
    user_item_normalized = normalize(user_item_csr, norm='l2', axis=0)
    return user_item_normalized.T.dot(user_item_normalized)

similarity_matrix = get_cosine_similarity_matrix(user_item_matrix)
sparse_info(similarity_matrix)

Размерности матрицы: (9067, 9067)
Ненулевых элементов в матрице: 21983224
Доля ненулевых элементов: 0.26740169371818234
Среднее значение ненулевых элементов: 0.04221750795841217
Максимальное значение ненулевых элементов: 1.0000016689300537
Минимальное значение ненулевых элементов: -1.0


In [91]:
from scipy.sparse import vstack, csr_matrix

def get_top_k_in_a_row(similarity_matrix, row, k, no_diagonal):
    lil_row = similarity_matrix[row]
    if no_diagonal:
        lil_row[0, row] = 0
    csr_row = lil_row.tocsr()
    csr_row.data[csr_row.data.argsort()[:-k]] = 0
    csr_row.eliminate_zeros()
    return csr_row

def get_top_k(similarity_matrix, k, no_diagonal=True):
    lil_similarity = similarity_matrix.tolil()
    top_k_similarity_matrix = get_top_k_in_a_row(lil_similarity, 0, k, no_diagonal)
    for row in range(1, lil_similarity.shape[0]):
        if len(lil_similarity.rows[row]) > 0:
            csr_row = get_top_k_in_a_row(lil_similarity, row, k, no_diagonal)
            top_k_similarity_matrix = vstack([top_k_similarity_matrix, csr_row])
    return top_k_similarity_matrix

top_k_similarity_matrix = get_top_k(similarity_matrix, 7)
sparse_info(top_k_similarity_matrix)

Размерности матрицы: (9067, 9067)
Ненулевых элементов в матрице: 63462
Доля ненулевых элементов: 0.000771945292771583
Среднее значение ненулевых элементов: 0.7470669150352478
Максимальное значение ненулевых элементов: 1.0
Минимальное значение ненулевых элементов: -0.04087800905108452


In [92]:
from scipy.sparse import coo_matrix

np.random.seed(123123)
train_percent = 0.1
user_item_matrix = user_item_matrix.tocoo()
train_split = np.random.choice(
    range(user_item_matrix.nnz),
    int(user_item_matrix.nnz * train_percent),
    replace=False
)
test_split = list(set(range(user_item_matrix.nnz)) - set(train_split))
train_matrix = coo_matrix(
    (
        user_item_matrix.data[train_split],
        (user_item_matrix.row[train_split], user_item_matrix.col[train_split])
    ),
    shape=user_item_matrix.shape
)
test_matrix = coo_matrix(
    (
        user_item_matrix.data[test_split],
        (user_item_matrix.row[test_split], user_item_matrix.col[test_split])
    ),
    shape=user_item_matrix.shape
)
print("Размер обучающей выборки:", train_matrix.nnz)
print("Размер тестовой выборки:", test_matrix.nnz)

Размер обучающей выборки: 10000
Размер тестовой выборки: 90004


In [93]:
from functools import reduce
def RMSE(matrix1: coo_matrix, matrix2: coo_matrix):
    arr = (matrix1 - matrix2).data
    return np.sqrt(reduce(lambda x, y: x + y**2, arr) / len(arr))

mean_rating_prediction = 0 * test_matrix
print("точность нашего baseline")
print(RMSE(mean_rating_prediction, test_matrix))

точность нашего baseline
1.05751754112


In [94]:
normalized_similarity = normalize(top_k_similarity_matrix.tocsr(), norm="l1", axis=0)
raw_predictions = train_matrix.dot(normalized_similarity).tocoo()
print("получили предсказаний всего")
print(len(raw_predictions.data))

filtered_predictions = binarize(test_matrix).multiply(raw_predictions)
print("оставили предсказаний для теста")
print(len(filtered_predictions.data))

получили предсказаний всего
62305
оставили предсказаний для теста
6614


In [95]:
print(RMSE(filtered_predictions, test_matrix))

1.04882443211
